# Binary star evolution and binary black holes

All the relevant information for the project are to be found in the pdf document present in the repo.
Note that you are assigned to project 1 (as the title said).

## Datasets 

Datasets are stored on Google Drive (link and description in the pdf document)

### Contacts

* Michela Mapelli <michela.mapelli@unipd.it>


In [1]:
import pandas as pd
import numpy as np
import string
import glob
import os
alphabet = list(string.ascii_lowercase)

In [15]:
alpha_values = [0.5, 1, 3, 5]
met_values = [0.02, 0.002, 0.0002, 0.004, 0.0004, 0.006, 0.008, 0.012, 0.0012, 0.016, 0.0016]

path = 'Users/tommaso/Desktop/MagistraleI/TheFormationOfBinaryBlackHoles_data/lab_data/stable_MT_vs_CE/A0.5'                     # use your path

li = []
for alpha in alpha_values:
    for met in met_values:
        
    
        for name in glob.glob(f'/Users/tommaso/Desktop/MagistraleI/TheFormationOfBinaryBlackHoles_data/lab_data/stable_MT_vs_CE/A{alpha}/*'):
            df = pd.read_csv(name, skiprows=2, header=0, sep=' ')
            df = df.loc[:,['col.0:ID','col.1:m1ZAMS/Msun','col.3:m1rem/Msun','col.4:m2rem/Msun',  'col.6:delay_time/Myr', 'col.7:sma/Rsun', 'col.8:ecc', 'col.21:CE']]
            met_array = np.ones((df.shape[0], 1))*met
            alpha_array = np.ones((df.shape[0],1))*alpha
            df['metallicity'] = met_array
            df['alpha'] = alpha_array
            
        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)


In [16]:
frame

,col.0:ID,col.1:m1ZAMS/Msun,col.3:m1rem/Msun,col.4:m2rem/Msun,col.6:delay_time/Myr,col.7:sma/Rsun,col.8:ecc,col.21:CE,metallicity,alpha
0,0_710,24.1323,9.5692,5.0020,141.1783,5.1703,0.258940,True,0.0200,0.5
1,0_1044,69.5474,28.4847,39.1186,9.2023,6.4328,0.055966,True,0.0200,0.5
2,0_1205,106.6977,39.9392,38.2094,2223.2639,35.8880,0.041435,False,0.0200,0.5
3,0_1289,74.3594,30.1523,37.4020,8.7628,6.0720,0.041342,True,0.0200,0.5
4,0_1702,116.6000,41.9604,42.9817,2636.2197,40.0410,0.079504,False,0.0200,0.5
...,...,...,...,...,...,...,...,...,...,...
7049345,4_1999631,25.2859,5.0281,9.1601,2595.3298,10.6030,0.222580,True,0.0016,5.0
7049346,4_1999663,24.3973,5.8065,13.2747,11488.7207,18.2060,0.073455,True,0.0016,5.0
7049347,4_1999908,34.5346,12.1456,15.5086,2722.8384,17.2320,0.018080,True,0.0016,5.0
7049348,4_1999924,46.2161,15.2529,23.8208,736.5745,15.9360,0.012796,True,0.0016,5.0


In [17]:
#MTCE_002_05_true = MTCE_002_05[(MTCE_002_05['col.21:CE'] == True)] 

In [18]:
#MTCE_002_05_true

In [19]:
#MTCE_002_05_false = MTCE_002_05[(MTCE_002_05['col.21:CE'] == False)] 
#MTCE_002_05_false